# 2D Spot Detection with ImageJ

## Discipline: Spot / object counting (SptCnt)

Task: Estimate the number of objects

This project illustrates the 2D counting of small vesicle like objects.

## The input images

The images were generated by [SIMCEP](http://www.cs.tut.fi/sgn/csb/simcep/tool.html), a widefield fluorescence microscopy biological images simulator.

First set paths to the input and ground-truth folders and create the output folder.

In [ ]:
IN_FOLDER = "/home/jovyan/data/in/"
GT_FOLDER = "/home/jovyan/data/gt/"
OUT_FOLDER = "/home/jovyan/data/out/"

!mkdir -p $OUT_FOLDER

The input images are in the [ome-tiff](https://docs.openmicroscopy.org/ome-model/6.0.1/ome-tiff/) format.

In [ ]:
import os
import tifffile
import matplotlib.pyplot
files = [i.path for i in os.scandir(IN_FOLDER) if i.is_file()]
in_paths_sources = [i for i in files if ('.tif' in i) ]
sources = []
for path in in_paths_sources:
    source = tifffile.imread(path)
    tifffile.imshow(source, cmap='gray')
    sources.append(source)
 

## The image analysis workflow

The ImageJ macro uses the Laplacian of Gaussian filter from [FeatureJ](https://imagej.net/FeatureJ) with the given standard deviation of the Gaussian derivative kernels and detects the minima with the given noise tolerance.

In [ ]:
from IPython.display import display, Code
code = Code(url='https://raw.githubusercontent.com/Neubias-WG5/W_SpotDetection-IJ/f2d8bd9a2ff74e56d9c80a98f0ab73838caca029/IJSpotDetection.ijm')
display(code)

## Running the workflow

The parameters for running the workflow are:
* --infolder: the folder containing the input images
* --gtfolder: the folder containing the ground truth data, only needed for benchmarking
* --outfolder: the folder into which the result images will be written

It has 3 switches that control the usage of the BIAflows server:
* --nodownload: the images are not downloaded from the server, they must already be in the input folder
* --noexport: the workflow results are only written into the output folder, but not uploaded to the server
* --nometrics: the metrics are neither computed nor uploaded to the server

If the analysis workflow has parameters these must also be passed. Their names can be found at the end of the file [descriptor.json](https://github.com/Neubias-WG5/W_SpotDetection-IJ/blob/master/descriptor.json).

The workflow parameters are:

* STD_DEV: The standard deviation of the Gaussian derivative kernels used for computing the second-order partial derivatives of the Laplacian. Must be larger than zero. The value is realated to the size of the spots that should be detected. The bigger the spots are the bigger the STD_DEV should be. See also the [documentation of FeatureJ](https://imagescience.org/meijering/software/featurej/laplacian/).

* NOISE: The noise tolerance for detecting the minima in the filtered image. Minima are ignored if they do not stand out from the surroundings by more than this value. See also the [ImageJ user guide](https://imagej.nih.gov/ij/docs/guide/146-29.html).

In [ ]:
STD_DEV = 2
NOISE = 2.5

In [ ]:
!python /app/wrapper.py  --ij_radius $STD_DEV --ij_noise $NOISE --infolder $IN_FOLDER --gtfolder $GT_FOLDER --outfolder $OUT_FOLDER --nodownload --noexport --nometrics

## Displaying the results

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from skimage.measure import label, regionprops
from skimage.color import label2rgb


files = [i.path for i in os.scandir(OUT_FOLDER) if i.is_file()]
paths_sources = [i for i in files if ('.tif' in i) ]
pointImages = []
index = 0
nrOfSpots = []
for path in paths_sources:
    fig, ax = plt.subplots(figsize=(20, 12))
    pointImage = tifffile.imread(path)
    label_image = label(pointImage)
    image_label_overlay = label2rgb(label_image, image=sources[index])
    ax.imshow(sources[index], cmap="gray")
    spotCount = 0
    for region in regionprops(label_image):
        (x,y) = region.centroid
        radius = region.equivalent_diameter/2.0
        rect = mpatches.Circle((y,x), 5, fill=False, edgecolor='red')
        ax.add_patch(rect)
        spotCount = spotCount + 1
    nrOfSpots.append(spotCount)
        
    index = index + 1
#    ax.set_axis_off()
    plt.tight_layout()
    plt.show()
    print("number of spots: " + str(spotCount))


Finally we display the number of spots for each image:

In [ ]:
import statistics
index = 0
for path in in_paths_sources:
    print(path + "\t" + str(nrOfSpots[index]))
    index = index + 1
    
print("Mean" + "\t" + str(statistics.mean(nrOfSpots)))
print("StdDev" + "\t" + str(statistics.stdev(nrOfSpots)))


In [ ]:
plt.boxplot(nrOfSpots)
plt.show()